In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
from datetime import datetime


In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data = pd.read_html(url)

In [ ]:
sp500 = data[0].iloc[:, [0,1,6,7]]
sp500.columns = ['ticker', 'name', 'date' , 'cik']

# Get rows where date is missing or not formatted correctly.
mask = sp500['date'].str.strip().str.fullmatch('\d{4}-\d{2}-\d{2}')
mask.loc[mask.isnull()] = False
mask = mask == False
sp500[mask].head()

In [ ]:
current = sp500.copy()
current.loc[mask, 'date'] = '1900-01-01'
current.loc[:, 'date'] = pd.to_datetime(current['date'])
current.loc[:, 'cik'] = current['cik'].apply(str).str.zfill(10)

In [ ]:
# Get the adjustments dataframe and rename columns
adjustments = data[1]
columns = ['date', 'ticker_added','name_added', 'ticker_removed', 'name_removed', 'reason']
adjustments.columns = columns

# Create additions dataframe.
additions = adjustments[~adjustments['ticker_added'].isnull()][['date','ticker_added', 'name_added']]
additions.columns = ['date','ticker','name']
additions['action'] = 'added'

# Create removals dataframe.
removals = adjustments[~adjustments['ticker_removed'].isnull()][['date','ticker_removed','name_removed']]
removals.columns = ['date','ticker','name']
removals['action'] = 'removed'

# Merge the additions and removals into one dataframe.
historical = pd.concat([additions, removals])
historical.head()

In [ ]:
"""
missing = current[~current['ticker'].isin(historical['ticker'])].copy()
missing['action'] = 'added'
missing = missing[['date','ticker','name','action', 'cik']]
missing.loc[:, 'cik'] = current['cik'].apply(str).str.zfill(10)
missing.head()
"""

In [ ]:
#sp500_history = pd.concat([historical, missing])
sp500_history = historical
sp500_history = sp500_history.sort_values(by=['date','ticker'], ascending=[False, True])
sp500_history = sp500_history.drop_duplicates(subset=['date','ticker'])
sp500_history

In [ ]:
starting = list(sp500['ticker'].values)

In [ ]:
sp500_history

In [ ]:
type(sp500_history[['date']].values[-1][0])

In [ ]:
def format_date(x):
    if(type(x)==pd._libs.tslibs.timestamps.Timestamp):
        return(x.strftime('%Y-%m-%d'))
    else:
        return(datetime.strptime(x, '%B %d, %Y'))
    #return datetime.strptime(x, '%B %d, %Y')

#sp500_history['date'].transform(format_date)

sp500_history_sorted = sp500_history.set_index(sp500_history['date'].transform(format_date)).sort_index(ascending=False)


In [ ]:
sp500_history_sorted

In [ ]:
sp500_history_sorted['ticker'][0]

In [ ]:
starting = list(sp500['ticker'].values)

In [ ]:
sp500_history_sorted

In [ ]:
starting

In [ ]:
np.sort(list(sp500['ticker'].values))

In [ ]:
#np.where(np.array(list(sp500['ticker'].values))=='Q')

In [ ]:
starting = list(sp500['ticker'].values)

sp500members_df = pd.DataFrame()
#missing Q

for d in (np.unique(sp500_history_sorted.index)[::-1]):
    #print(d)
    d_subset = sp500_history_sorted.loc[[d]].sort_values(by='action',ascending=True)
    print(d_subset)
    sp500members_df = pd.concat([sp500members_df,pd.DataFrame([[starting]],index=[d],columns=['members'])],axis=0)
    
    for d_ in range(0,len(d_subset)):
        d_data = d_subset.iloc[[d_]]

        ticker = d_data['ticker'][0]
        print(ticker)
        #print(starting)
        if(d_data['action'][0]=='added'):
            #print("removing",ticker)
            remove = np.where(np.array(starting)==ticker)
            if(len(remove[0])==0):
                pass
            else:
                starting.pop(remove[0][0])
            
        else:
            #print(starting)
            #print("adding", ticker)
            starting.extend([ticker])
    print(len(starting))
    #print(starting)
    pd.DataFrame(starting)


In [ ]:
sp500members_df.to_csv("sp500_historical_members.csv")